# Create and visualize datasets

In [1]:
import numpy as np, pandas as pd
import ta

# import sys
# sys.path.append('../scripts/')

from backtest_utils import *

### Load from raw data

In [2]:
complete_data = pd.read_csv('data/test.csv')
rolling_pca_data = pd.read_csv('data/rolling_pca.csv', index_col=0, parse_dates=['date'])

### Transform raw data into proper signal-full data

In [3]:
df, train, test = get_workable_data(complete_data)

df_benchmark = pd.DataFrame(df.Equi_weighted).copy()

df_prices = df.iloc[:,:6].copy().fillna(method='ffill') #(2)

df_prices = np.subtract(df_prices.pct_change(),df_benchmark.pct_change())
for col in df_prices.columns :
    df_prices[col + '_ema_20'] = df_prices[col].ewm(alpha=1/20, ignore_na=True).mean()
    df_prices[col + '_ema_60'] = df_prices[col].ewm(alpha=1/60, ignore_na=True).mean()
    del df_prices[col]
    
df_macros = df.iloc[:,6:15].copy().fillna(method='ffill') #(2)

df_vol_signal = ema_and_signal(df_macros.iloc[:,:3], [20,30,40],[60,80,100])
df_surprises_signal = ema_and_signal(df_macros.iloc[:,-4:], [20,30,40],[60,80,100])

for col in df_macros.columns :
    df_macros[col + '_ema_20'] = df_macros[col].rolling(window=20).mean()
    df_macros[col + '_ema_60'] = df_macros[col].rolling(window=60).mean()
    del df_macros[col]
df_macros = (df_macros-df_macros.mean())/df_macros.std()

#Only prices, transformed through relative return and exponential mooving average (20 and 60 days)
dataset1 = pd.concat([df_prices, df_benchmark], axis=1).dropna()
#Prices, transformed as above, and macros transformed in simple moving average (20 and 60 days), then standardised
dataset2 = pd.concat([df_prices, df_macros, df_vol_signal, df_surprises_signal, df_benchmark], axis=1).dropna()
#Only macros regarding volatility (UX and Barclays indicator), transformed as above
dataset3 = pd.concat([df_macros.iloc[:, :6], df_vol_signal, df_benchmark], axis=1).dropna()
#Only macros regarding economic surprises, transformed as above
dataset4 = pd.concat([df_macros.iloc[:, -8:], df_surprises_signal, df_benchmark], axis=1).dropna()

df, train, test = get_workable_data(complete_data)

n_stoch = 20
d_stoch = 10

df_benchmark = pd.DataFrame(df.Equi_weighted).copy()

df_prices = df.iloc[:,:6].copy().fillna(method='ffill') #(2)

for col in df_prices.columns:
    df_prices[col] = ta.momentum.stoch_signal(df_prices[col],df_prices[col],df_prices[col],\
                                              n=n_stoch, d_n=d_stoch, fillna=False)/100
df_macros = df.iloc[:,6:15].copy().fillna(method='ffill') #(2)
df_macros = (df_macros-df_macros.mean())/df_macros.std()

#Macros standardized and prices transformed in stochastic
dataset5 = pd.concat([df_prices, df_macros, df_benchmark], axis=1).dropna()

#PCA of previous dataset, rolling (no forward-looking), using 4 factors (75% of variance)
dataset6 = pd.concat([rolling_pca_data, df_benchmark], axis=1).dropna()

#New macros standardized with prices still in stochastic
new_macros = pd.read_csv('data/macroindicators_ms.csv', index_col = 0, parse_dates=['Date'])
del new_macros['Trailing 12M Gross Margin \n(SX5E Index)']
new_macros = new_macros.fillna(method = 'ffill')
new_macros = (new_macros-new_macros.mean())/new_macros.std()

dataset7 = pd.concat([df_prices, new_macros, df_benchmark], axis=1).dropna()

for df in [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7]:
    df.Equi_weighted = df.Equi_weighted/df.Equi_weighted[0]*1000

### To write

In [4]:
# dataset1.to_csv('data/dataset1.csv')
# dataset2.to_csv('data/dataset2.csv')
# dataset3.to_csv('data/dataset3.csv')
# dataset4.to_csv('data/dataset4.csv')
# dataset5.to_csv('data/dataset5.csv')
# dataset6.to_csv('data/dataset6.csv')
# dataset7.to_csv('data/dataset7.csv')

### To load

In [5]:
dataset1 = pd.read_csv('data/dataset1.csv', index_col=0)
dataset2 = pd.read_csv('data/dataset2.csv', index_col=0)
dataset3 = pd.read_csv('data/dataset3.csv', index_col=0)
dataset4 = pd.read_csv('data/dataset4.csv', index_col=0)
dataset5 = pd.read_csv('data/dataset5.csv', index_col=0)
dataset6 = pd.read_csv('data/dataset6.csv', index_col=0)
dataset7 = pd.read_csv('data/dataset7.csv', index_col=0)